# Direct B2C Revenue

In [0]:
CREATE or replace view dev.bi.s12n_estimated_b2c_direct_revenue_mgtv  AS
WITH base_s12n_launch_dts as (
    SELECT 
        base_phoenix_specialization_id, 
        min(phoenix_specialization_display_page_launch_ts) as first_s12n_display_page_launch_ts
    FROM prod.gold.phoenix_specializations_vw
    GROUP by 1
  ),

s12n_table AS
(
SELECT 
ps.base_phoenix_specialization_id,
ps.phoenix_specialization_id,
ps.phoenix_specialization_slug,
ps.phoenix_specialization_name,
bsl.first_s12n_display_page_launch_ts::DATE AS launch_date
FROM
prod.gold.phoenix_specializations_vw ps
JOIN
base_s12n_launch_dts bsl 
USING (base_phoenix_specialization_id)
),

-- aggregate monthly B2C direct revenue for every S12N by month (we want to aggregate across all transactions separately)

-- get B2C Direct monthly revenue
direct_monthly_aggregated AS 
(
SELECT 
ebrr.rev_rec_month_start_dt,
ebrr.attributed_content_item_id as underlying_product_item_id,
'b2c_direct' AS transaction_type,
SUM(ebrr.rev_rec_amount_usd_estimate) AS rev_rec_amount_usd_estimate
FROM
prod.gold_base_restricted.estimated_combined_revenue_recognition ebrr
where is_cplus_transaction = false
and rev_rec_month_start_dt < to_date(date_trunc('month',current_date))
and rev_rec_month_start_dt >=to_date(date_trunc('month', add_months(current_date, -2)))
GROUP BY 1, 2, 3
)
SELECT 
dma.rev_rec_month_start_dt,
dma.underlying_product_item_id,
st.base_phoenix_specialization_id,
st.phoenix_specialization_id,
st.phoenix_specialization_slug,
st.phoenix_specialization_name,
st.launch_date,
dma.transaction_type,
dma.rev_rec_amount_usd_estimate
FROM
direct_monthly_aggregated dma
JOIN
s12n_table st
ON
dma.underlying_product_item_id = st.phoenix_specialization_id 


# C+ Revenue

In [0]:
CREATE or replace view dev.bi.s12n_estimated_cplus_revenue_mgtv AS
WITH base_s12n_launch_dts as (
    SELECT 
        base_phoenix_specialization_id, 
        min(phoenix_specialization_display_page_launch_ts) as first_s12n_display_page_launch_ts
    FROM prod.gold.phoenix_specializations_vw
    GROUP by 1
  ),

s12n_table AS
(
SELECT 
ps.base_phoenix_specialization_id,
ps.phoenix_specialization_id,
ps.phoenix_specialization_slug,
ps.phoenix_specialization_name,
bsl.first_s12n_display_page_launch_ts::DATE AS launch_date
FROM
prod.gold.phoenix_specializations_vw ps
JOIN
base_s12n_launch_dts bsl 
USING (base_phoenix_specialization_id)
),

cplus_monthly_aggregated AS (
SELECT 
ecrr.rev_rec_month_start_dt,
ecrr.attributed_content_item_id as underlying_product_item_id,
'cplus' as transaction_type,
SUM(ecrr.rev_rec_amount_usd_estimate) AS rev_rec_amount_usd_estimate
FROM
prod.gold_base_restricted.estimated_combined_revenue_recognition ecrr
where is_cplus_transaction and attribution_logic = 'EBRS' 
and rev_rec_month_start_dt < to_date(date_trunc('month',current_date))
and rev_rec_month_start_dt >=to_date(date_trunc('month', add_months(current_date, -2)))
group by 1, 2, 3
)
SELECT 
cma.rev_rec_month_start_dt,
cma.underlying_product_item_id,
st.base_phoenix_specialization_id,
st.phoenix_specialization_id,
st.phoenix_specialization_slug,
st.phoenix_specialization_name,
st.launch_date,
cma.transaction_type,
cma.rev_rec_amount_usd_estimate
FROM
cplus_monthly_aggregated cma
JOIN
s12n_table st
ON
cma.underlying_product_item_id = st.phoenix_specialization_id;


# Combined Revenue

In [0]:
CREATE or replace view dev.bi.s12n_estimated_total_b2c_revenue_mgtv AS 
(
-- aggregate cplus revenue to base S12N ID level
WITH cplus AS
(
  SELECT 
    base_phoenix_specialization_id,
    launch_date,
    rev_rec_month_start_dt,
    SUM(rev_rec_amount_usd_estimate) as revenue
  FROM
  dev.bi.s12n_estimated_cplus_revenue_mgtv
  GROUP BY 1, 2, 3
),

-- do the same with direct transactions
direct AS
(
  SELECT 
    base_phoenix_specialization_id,
    launch_date,
    rev_rec_month_start_dt,
    SUM(rev_rec_amount_usd_estimate) as revenue
  FROM
  dev.bi.s12n_estimated_b2c_direct_revenue_mgtv
  GROUP BY 1, 2, 3
),

-- union both tables
union_revenue AS 
(
  SELECT * FROM cplus
  UNION ALL
  SELECT * FROM direct
)

SELECT
    base_phoenix_specialization_id,
    launch_date,
    rev_rec_month_start_dt,
    SUM(revenue) as b2c_revenue
FROM union_revenue
GROUP BY 1, 2, 3
)

%md
# Deleting last 2 months of data

In [0]:
-- drop table if exists dev.bi.s12n_estimated_total_b2c_revenue_mgtf;
-- create table dev.bi.s12n_estimated_total_b2c_revenue_mgtf as
-- select * from dev.bi.s12n_estimated_total_b2c_revenue_mgtv;

DELETE FROM dev.bi.s12n_estimated_total_b2c_revenue_mgtf
WHERE rev_rec_month_start_dt >= to_date(date_trunc('month', add_months(current_date, -2)))


num_affected_rows
1689


# Appending the latest last 2 months of data

In [0]:
INSERT INTO dev.bi.s12n_estimated_total_b2c_revenue_mgtf
select * from dev.bi.s12n_estimated_total_b2c_revenue_mgtv

num_affected_rows,num_inserted_rows
3430,3430


In [0]:
select year(rev_rec_month_start_dt) as yy,sum(b2c_revenue) 
from dev.bi.s12n_estimated_total_b2c_revenue_mgtf 
group by 1 order by 1


yy,sum(b2c_revenue)
2020,154590879.0254891901
2021,211984197.1846560071
2022,268411151.4224615748
2023,337687842.6292258343
2024,362605278.0038679186
2025,157646456.6938899522
